Amoeba-based superpixel partitioning of multispectral images into elementary, uniform, connected units
---

Scene segmentation is a difficult task because of the high complexity of images, where complexity refers to the large variety of pictorial representations of objects with the same semantic meaning and also to the extensive amount of available details. We introduce a new approach inspired by the SLIC methodology that implicitly enforces connectivity and efficiently generates compact, connected, and nearly uniform superpixels. The new formulation will introduce a different neighbourhood structure that implicitly encode spatial proximity and contiguity in it. 


#### Rationale

The problem of segmenting an image into semantically meaningful units is a common one in image processing [[SM00]](#SM00). Indeed, visual information extraction is often regarded as a segmentation issue [[FH04]](#FH04). Scene segmentation captures the local redundancy in the data, by reducing noise and variability, but aggregating pixels into segments often entails a decision that is unrelated to the final task. The goal is then to perform this decision in a conservative way to minimize the risk of merging unrelated pixels from different objects. As a pragmatic alternative, **over-segmentation forms a one-to-many partitioning of scene features into smaller segments of distinct spectra**. It seems indeed natural, and presumably more efficient, to **work with perceptually meaningful entities obtained from low-level grouping processes instead of the pixel representation**. In that context, **superpixels obtained from conservative over-segmentation are a common pre-processing step for recovering image features** [[RM03](#RM03), [LDM07](#LDM07), [MPWMJ08](#MPWMJ08)]. 

We introduce an algorithm that _(i)_ works essentially like a _kmeans_ based local clustering of pixels, but _(ii)_ enforces connectivity, so that it can efficiently generate compact, connected, and nearly uniform superpixels. This approach is based on the **estimation of amoeba-like neighborhoods around selected cluster centers that exploit the connections between successive image pixels along geodesic paths in the image**. The resulting superpixels capture the spatial/spectral redundancy in images and greatly reduce the complexity of subsequent image processing tasks [[Hanbury08]](#Hanbury08). They provide convenient primitives from which to compute local image features when objects present in the scene have diverse scales or when they are not known in advance [[BBI08]](#BBI08).

#### Original SLIC segmentation

Whereas the idea of operating on atomic, homogeneously colored or textured regions is not new, the popular term **superpixels** has been coined recently. In practice, any segmentation of an image into contiguous regions could produce superpixels. However, for many image processing tasks, compact and highly uniform superpixels that respect image boundaries are desired.
The SLIC algorithm proposed in [[ASSLFS12]](#ASSLFS12) partitions an image into a desired number of regular, compact superpixels with a low computational overhead.
It is essentially a **_kmeans_ algorithm that partitions a color image into a desired number of regular, compact superpixels with a low computational overhead**. Namely, it locally clusters pixels in the combined 5D color ($(L, a, b)$ values of the CIELAB color space) and image plane ($(x, y)$ pixel coordinates) space. Note that the reason why CIELAB color space is chosen is that it is perceptually uniform for small color distance.  

The SLIC algorithm takes as input a desired number of approximately equally-sized superpixels $K$ and begins by sampling $K$ regularly spaced cluster centers $C_k = [L_k, a_k, b_k, x_k, y_k]^T$ at regular grid intervals $S$ (depending on $K$ and the number $N$ of pixels in the image: $S = \sqrt{N/K}$). In practice, for roughly equally sized superpixels there would be a superpixel center at every grid interval $S$. Since the spatial extent of any superpixel is approximately $S^2$ (the approximate area of a superpixel), it is assumed that pixels that are associated with this cluster center lie within a $2S \times 2S$ area around the superpixel center on the $(x, y)$ plane. This becomes the search area for the pixels nearest to each cluster center. Instead of directly using the Euclidean distance in the 5D space, **a _"perceptual closeness"_ measure $D_s$ that considers superpixel size is introduced to control the compactness of superpixels** [[ASSLFS12]](#ASSLFS12):
$$
D_s ({\mathbf x}_i, {C_k}) = d_{lab} ({\mathbf x}_i, {C}_k) + \frac{m}{S} \cdot {d}_{xy} ({\mathbf x}_i, {C}_k),
\qquad (1)
$$
defined as the (weighted) sum of the spectral distance $d_{lab}$ in the color space:
$$
d_{lab} ({\mathbf x}_i, {C}_k) = \sqrt{(L_k - L_i)^2 + (a_k - a_i)^2 + (b_k - b_i)^2} 
\qquad (2a)
$$
and the spatial distance $d_{xy}$ in the plane (normalized by the grid interval $S$):
$$
d_{xy} ({\mathbf x}_i, {C_k}) = \sqrt{(x_k - x_i)^2 + (y_k - y_i)^2} .
\qquad (2b)
$$
of a pixel ${\mathbf x}_i=[ L_i, a_i, b_i, x_i, y_i ]^T$ to any cluster center $C_k$. Here, $m$ is introduced to control the compactness of superpixels (the greater the value of $m$, the more spatial proximity is emphasized and the more compact the cluster). 
**This distance is used to cluster together preceptually close pixels in an iterative manner**. Namely, each pixel ${\mathbf x}_i$ is associated with the nearest cluster center $C_k$  (_w.r.t_ the distance $D_s$) whose search area overlaps this pixel. After all the pixels are associated with their nearest cluster center, new centers are  computed as the barycenters of all the pixels belonging to the updated clusters. This process is iterated till convergence to produce approximately equally-sized superpixels. 

A post-processing of the clusters output by the _kmeans_ clustering is necessary to enforce connectivity. Indeed, the distance $D_s$ (based on composite Euclidean distance), while being easy to calculate, **does not take into account the image intensity values between two image pixels and thus ignores connectivity**: a pixel belonging to an object distinct from the cluster center but with similar spectral values, while separated by other intermediate distinct structures, can still reach low $D_s$ and be considered as _"perceptually close"_, hence the superpixels can be ripped apart. 

 
#### Extended amoeba-based SLIC segmentation

Generally, making an appropriate choice of homogeneity criterion is critical to the success of any region-based segmentation procedure, especially for multispectral images, where the criterion is highly dependent on the choice of the (spectral and spatial) closeness measure [[FH04]](#FH04). Our algorithm is based on the **estimation of amoeba-like neighborhoods around the cluster centers** [[LDM07](#LDM07), [GS09a](#GS09)]. **The amoeba construction exploits the connections between successive image pixels along the so-called geodesic paths** to define a related measure [[DP06](#DP06), [Soille08](#Soille08)]. It describes **similarity relationships based on pairwise differences between neighboring pixels** (_e.g._, using the connections between successive image pixels along geodesic paths) [[BM06](#BM06), [GDS10](#GDS10)].

<table>
<tr>
<td><kbd><img src="img/excerpt1.png" alt="input excerpt#1" width="250"> </kbd></td>
<td><kbd><img src="img/exceprt1-amoeba-superpixels.png" alt="amoeba superpixels" width="250"> </kbd></td>
<td><kbd><img src="img/excerpt1-mean-amoeba-approximations.png" alt="mean amoeba approximations" width="250"> </kbd></td>
</tr>
<tr>
<td align="center" width="250">input excerpt</td>
<td align="center" width="250">amoeba superpixels (more scales displayed)</td>
<td align="center" width="250">amoeba mean-averaged approximations (more scales displayed)</td>
</tr>

</table>

A new _"perceptual closeness"_ measure between ${\mathbf x}_i$ and $C_k$ is formulated as the minimal length of the  shortest path ${\cal P} = (p_0={\mathbf x}_i, p_1, \cdots, p_n=C_k)$ joining them [[GS08](#GS08), [GS08](#GS09)]. $D_a$ is estimated as the **combined spatial/spectral cost of "traveling"** from ${\mathbf x}i$ to $C_k$ in the image: 
$$
D_a ({\mathbf x}_i,C_k) = \min_{\cal P} \{ L_a({\cal P}) ({\mathbf x}_i,C_k) \}
\qquad (3)
$$
where
$$
L_a({\cal P}) ({\mathbf x}_i, C_k) = \sum_{j=0}^{j=n-1} d_{lab}(p_j,p_{j+1}) + \alpha \cdot d_{xy}(p_j,p_{j+1}) + \beta \cdot d_g(p_j,p_{j+1})
\qquad (4)
$$
with $d_{lab}$ and $d_{xy}$ as before. 
Similarly to $m$ previously, $\alpha$ quantifies the **relative influences of the spectral proximity _w.r.t_ the spatial one (simply computed as the local Euclidean distance) in the estimation of the perceptual measure**: the higher $\alpha$, the more spectral closeness is emphasized. 
In addition, we **incorporate local information about the meaningfull structures present in the image using the cost induced by the gradient structure tensor of the image** $d_g(p_j,p_{j+1})$ [[GDS10]](#GDS10). This way, high gradient pixels act like barriers in the amoeba propagation: the lower the gradient values from $p_j$ to $p_{j+1}$, the higher the probability they belong to the same amoeba. Say it otherwise, the higher $\beta$, the less probable amoeba superpixels will cross regions of high gradient. Whilst using more information, we will need to deal with the problem of how to weight and combine different kinds of information as  $\alpha$ and $\beta$ quantify the relative influences of spectral, spatial and surface proximities in the estimation of the perceptual measure: the higher $\alpha and \beta$ are, the less compact the clusters are [[GS08](#GS08), [GS08](#GS09)].  

#### Algorithm implementation

Given a user-specified amount of superpixels (say $K$), the algorithm first puts, likewise the original SLIC [[ASSLFS12]](#ASSLFS12), some seeds roughly in a lattice structure on the image along with small disturbance in order to avoid the placement on strong intensity boundaries. The seeds serve as initial estimates of the superpixel centers. The location of the centers and shape of each superpixel keep changing in turn as the algorithm runs. 

<img src="algorithm.png" alt="algorithm amoeba superpix” width=“700">

In computing geodesic distances in a local $(2S \times  2S)$  exploration neighborhoods 
around each cluster center, **the amoeba approach employs priority queues**, whose complexity is $O(w \log(w))$ in the size of this neighborhood  $w = 4S^2 = 4N/K$ (with: $N$ the total number of pixels). Therefore the estimation of one superpixel area only is $O((N/K) \log(N/K))$ and the final complexity is $O (N \log(N/K))$:
\begin{equation}
 O(\textsf{cardinal}(\textsf{superpixels}) \cdot \textsf{cost}(\textsf{superpixel}))  \\
= O (K \cdot (N/K) \cdot \log(N/K)) 
= O (N \cdot \log(N/K)) 
\end{equation}
The process that consists in associating pixels with the nearest cluster center and recomputing the cluster center is iteratively repeated until convergence. **By dividing the image into distinct, non-overlapping regions, each containing a number of pixels with some consistent and perceptually meaningful set of properties, a segmentation map is finally formed**.
